This notebook speciate total edgarv5 nmvoc to MOZART mechanism using the fractional nmvoc contribution 
obtained from the edgar4.3.2  VOCs mapping to MOZART. (speciated vocs monhtly emission by sector for 2010- 17sectors in total).
(NOTE: speciation is provided for all sectors having NMVOC in edgarv5 using lumped sectors to match edgarv4.3.2 sectors. All sector with NMVOC emissions are speciated except for manure management (MNM, ipcc1996:4B), which was not included in the vocs speciation of edgarv4.3.2).

In [1]:
import xarray as xr
import os
import numpy as np
import pandas as pd

### Speciate NMVOC

In [2]:
#get total nmvoc
nmvoc_pth = '/geos/d21/s1878599/edgarv5_process/mapped_monthly_v50_2015_NMVOC_.0.1x0.1.nc'
vocs_dir = '/geos/d21/s1878599/edgarv5_process/monthly_nmvocs4.3.2_fractional_MOZART/'
save_dir='/geos/d21/s1878599/edgarv5_process/monthly_all_sectors/'

In [3]:
nmvoc=xr.open_dataset(nmvoc_pth)

In [4]:
nmvoc

<xarray.Dataset>
Dimensions:           (lat: 1800, lon: 3600, time: 12)
Coordinates:
  * lat               (lat) float32 -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
  * lon               (lon) float32 0.05 0.15 0.25 0.35 ... 359.75 359.85 359.95
  * time              (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    AWB               (time, lat, lon) float32 ...
    ENE               (time, lat, lon) float32 ...
    IND               (time, lat, lon) float32 ...
    PPA               (time, lat, lon) float32 ...
    PRO               (time, lat, lon) float32 ...
    RCO               (time, lat, lon) float32 ...
    SWD               (time, lat, lon) float32 ...
    TNR-Aviation-CDS  (time, lat, lon) float32 ...
    TNR-Aviation-CRS  (time, lat, lon) float32 ...
    TNR-Aviation-LTO  (time, lat, lon) float32 ...
    TNR-Other         (time, lat, lon) float32 ...
    TNR-Ship          (time, lat, lon) float32 ...
    REF-TRF           (time, lat, lon) float32 ...
    TRO               (time, lat, lon) float32 ...
    FFF               (time, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    title:             Monthly Mean (Oct) Emissions of NMVOC - 
    institution:       European Commission, Joint Research Centre
    source:            http://edgar.jrc.ec.europa.eu/
    history:           Created from original data (0.1x0.1 degrees) using IDL...
    references:        European Commission, Joint Research Centre (JRC)/Nethe...
    copyright_notice:  Reproduction of the data is authorized, except for com...
    contact:           JRC-EDGAR@ec.europa.eu

In [5]:
for f in os.listdir(vocs_dir):
    voc_name=f.split('_')[4]
    print(voc_name)
    voc_fr=xr.open_dataset(vocs_dir+f)
    
    #REF dn TRF need to be aggregrated in the VOC
    voc_fr['REF-TRF']=voc_fr['REF']+voc_fr['TRF']
    
    #speciate by fraction.
    ds=nmvoc*voc_fr
    ds.attrs['title']='Monthly emissions of ' + voc_name + ' kg m-2 s-1'

    ds.to_netcdf(save_dir+'monthly_v50_2015_'+voc_name+'_.0.1x0.1.nc',format='NETCDF3_64BIT') #save new file.

CH2O
TOLUENE
BIGALK
XYLENES
BIGENE
CH3OH
MEK
C2H2
CH3CHO
C3H8
CH3COCH3
HCOOH
C2H6
C2H5OH
CH3COOH
BENZENE
C3H6
C2H4


In [6]:
#check sample output if  it is formatted correctly
t=xr.open_dataset(save_dir+'monthly_v50_2015_BIGALK_.0.1x0.1.nc')

In [7]:
t

<xarray.Dataset>
Dimensions:           (lat: 1800, lon: 3600, time: 12)
Coordinates:
  * lat               (lat) float32 -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
  * lon               (lon) float32 0.05 0.15 0.25 0.35 ... 359.75 359.85 359.95
  * time              (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    AWB               (time, lat, lon) float32 ...
    ENE               (time, lat, lon) float32 ...
    IND               (time, lat, lon) float32 ...
    PPA               (time, lat, lon) float32 ...
    PRO               (time, lat, lon) float32 ...
    RCO               (time, lat, lon) float32 ...
    SWD               (time, lat, lon) float32 ...
    TNR-Aviation-CDS  (time, lat, lon) float32 ...
    TNR-Aviation-CRS  (time, lat, lon) float32 ...
    TNR-Aviation-LTO  (time, lat, lon) float32 ...
    TNR-Other         (time, lat, lon) float32 ...
    TNR-Ship          (time, lat, lon) float32 ...
    REF-TRF           (time, lat, lon) float32 ...
    TRO               (time, lat, lon) float32 ...
    FFF               (time, lat, lon) float32 ...
Attributes:
    title:    Monthly emissions of BIGALK kg m-2 s-1

### Check if sum of NMVOC is right

In [8]:
#get list of vocs.
v=[]
for f in os.listdir(vocs_dir):
    voc_name=f.split('_')[4]
    v.append(voc_name)

#get all vocs files as ordered dictionary of xarray datasets.
vocs={}
for f in os.listdir(save_dir):
    vname=f.split('_')[3]
    if vname in v:
        print(vname)
        vocs.update({vname:xr.open_dataset(save_dir+f)})   

C2H5OH
CH3OH
CH3COOH
MEK
TOLUENE
BIGENE
CH3COCH3
CH2O
C2H6
BENZENE
XYLENES
HCOOH
C3H8
C2H2
C3H6
CH3CHO
C2H4
BIGALK


In [9]:
vocs

{'C2H5OH': <xarray.Dataset>
 Dimensions:           (lat: 1800, lon: 3600, time: 12)
 Coordinates:
   * lat               (lat) float32 -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
   * lon               (lon) float32 0.05 0.15 0.25 0.35 ... 359.75 359.85 359.95
   * time              (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
 Data variables:
     AWB               (time, lat, lon) float32 ...
     ENE               (time, lat, lon) float32 ...
     IND               (time, lat, lon) float32 ...
     PPA               (time, lat, lon) float32 ...
     PRO               (time, lat, lon) float32 ...
     RCO               (time, lat, lon) float32 ...
     SWD               (time, lat, lon) float32 ...
     TNR-Aviation-CDS  (time, lat, lon) float32 ...
     TNR-Aviation-CRS  (time, lat, lon) float32 ...
     TNR-Aviation-LTO  (time, lat, lon) float32 ...
     TNR-Other         (time, lat, lon) float32 ...
     TNR-Ship          (time, lat, lon) float32 ...
     REF-TRF           (time, lat, l

In [10]:
tot_vocs=sum(vocs.values())

In [11]:
tot_vocs

<xarray.Dataset>
Dimensions:           (lat: 1800, lon: 3600, time: 12)
Coordinates:
  * time              (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * lon               (lon) float32 0.05 0.15 0.25 0.35 ... 359.75 359.85 359.95
  * lat               (lat) float32 -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
Data variables:
    AWB               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ENE               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    IND               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PPA               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PRO               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    RCO               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SWD               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    TNR-Aviation-CDS  (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    TNR-Aviation-CRS  (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    TNR-Aviation-LTO  (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    TNR-Other         (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    TNR-Ship          (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    REF-TRF           (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    TRO               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    FFF               (time, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [12]:
# check if each sector sum is right
for var in nmvoc.data_vars.values():
    v=var.name
    print(v)
    xr.testing.assert_allclose(nmvoc[v], tot_vocs[v],rtol=1e-6,atol=1e-7)  

AWB
ENE
IND
PPA
PRO
RCO
SWD
TNR-Aviation-CDS
TNR-Aviation-CRS
TNR-Aviation-LTO
TNR-Other
TNR-Ship
REF-TRF
TRO
FFF
